# **EDA y primer modelo - Grupo03**

> Bloque con sangría


### **Authors**
- César López Mantecón - 100472092
- Manuel Gómez-Plana Rodríguez - 100472310

### **Repositorio**
Esta práctica se ha llevado a cabo en [este repositorio de github](https://github.com/CLopMan/aprendizajeAutomatico-G03)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Índice**
...

## **Introducción**
En este *notebook* se recoge el Análisis de Datos Exploratorio (EDA, por sus siglas en inglés[1]) además de la selección del primer modelo. Comenzaremos por el EDA. Este proceso constará de los siguientes pasos: ajuste de datos, determinación de características e instancias, análisis de variables e instancias (existencia de valores nulos) y modelización del problema.

Tras el EDA, haremos un análisis de la evaluación *outer* e *inner* junto con la decisión de las métricas usadas. Luego, decidiremos el método de escalado más adecuado usando el KNN como algoritmo, para luego probar varios modelos para poder decidir cual es el mejor.

Ahora, comencemos con el EDA.

---

[1]: IBM, "Análisis de datos exploratorio". IBM. https://www.ibm.com/es-es/topics/exploratory-data-analysis (acceso: 28 de febrero de 2024)

# **Contexto del problema**

Creo que podría quedar bien decir qué pretendemos hacer, cuál es la variable objetivo y demás.

# **EDA - 1. Análisis preliminar**

Usaremos el módulo pandas para analizar los datos. Esto nos permitirá usar la herramienta de dataframes con el fin anterior. Primero, mostraremos las primeras y últimas filas del dataframe para hacernos una idea preliminar de los datos. Además, haremos un recuento de instancias para ver las dimensiones del conjunto de datos.


In [ ]:
import pandas as pd

raw_data = pd.read_csv("wind_ava.csv.gz", compression='gzip') # lectura del fichero en bruto
print(raw_data.head()) # primeras filas
print(raw_data.tail()) # últimas filas
print("========DIMENSIONES========")
print("nº filas = " + str(len(raw_data)) + "\nnº columnas = " + str(len(csv.columns)))

FileNotFoundError: [Errno 2] No such file or directory: 'wind_ava.csv.gz'

Observamos que se trata de un conjunto de datos con **4748 instancias y 552 features** a priori numéricas. Para obtener más información del mismo haremos uso del método df.info() y del atributo df.dtypes.


In [ ]:
print(raw_data.info())
print(raw_data.dtypes)



Gracias a esta información, sabemos 551 de las 552 features son de tipo numérico (concretamente, números en coma flotante de doble precisión). Además, vemos que datetime es la única característica no numérica. No obstante, al modelar una marca de tiempo se podría convertir a numérica si fuera necesario. Observamos también que, a escepción de energy y datetime, las variables se pueden relacionar fácilmente con la localización a la que hacen referencia a través del nombre de la columna en el dataframe. Con esto, concluímos esta sección con un breve resumen de las conclusiones extraídas.

- El conjunto de datos cuenta con 4748 instancias.
- Cada instancia cuenta con 551 características del tipo numérico y un objeto str fácilmente convertible a numérico.
- Los datos están ligados a una localización fácilmente observada en el nombre de la columna.



# **EDA - 2.Ajuste y primer análisis de datos**

En este apartado haremos un pequeño preprocesado de los datos para eliminar la instancias que no son relevantes para el problema propuesto. Para ello, eliminaremos todos los datos que no hagan referencia a la localización "Sotavento". Además, volvemos a imprimir información sobre el dataframe transformado para observar los cambios después de esta operación.


In [ ]:
sotavento = raw_data.filter(regex='13$|energy|datetime') # eliminacion de instancias no referentes a Sotavento

print(sotavento.info())
print(sotavento.dtypes)

Comprobamos que este feature space es un espacio 24-dimensional con 4748 instancias. Al filtrar otras localizaciones hemos reducido drásticamente el número de variables a tener en cuenta. En la salida del código anterior también podemos observar que **no existen valores nulos en ninguna instancia** ya que el valor `Non-Null Count es igual al número de instancias para todas las variables [*].

[] *Nota: téngase en cuenta que, dependiendo del tamaño de la pantalla y el visualizador del notebook* la salida del código anterior puede haber sido truncada. Recomendamos copiar la salida a un visualizador de texto cualquiera para poder observarla sin problemas.*


## **EDA - 3. Análisis de variables e Instancias**

A continuación, estudiaremos el conjunto de datos para comprobar la existencia de relaciones entre variables, con el fin concer en profundidad el conjunto de datos, así como de de limpiarlo de variables que no sean relevantes para el estudio. Para ello, analizaremos en profundidad cada instancia con el fin de detectar valores atípicos y la existencia de correlación entre las mismas o con la variable objetivo. Primero, extraeremos un conjunto de estadísticas descriptivas del conjunto de datos:

In [ ]:
print(sotavento.describe())

NameError: name 'csv' is not defined

En la tabla anterior podemos observar la media; desviación estándar; mínimo; valores del primer, segundo y tercer cuartil; y máximo de cada una de las features. Para complementar estos datos se han generado diferentes gráficos de cada variable para tener una idea de la distribución de los datos. Para la generación de gráficas nos valdremos de los módulos `mathplotlib` y `seaborn`.

### **Diagrama de barras** (Aquí metería diagrama de distribuciones de los datos, más que diagramas de barras.)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(10, 6))

columns = list(sotavento.columns)
columns.remove('datetime') # no relevante, marca de tiempo

for _ in columns:
    sns.histplot(sotavento[_], kde=True)
    plt.title(str(_))
    plt.show()

### **Existencia de valores atípicos**

### **Estudio de correlaciones**

Para representar la correlación entre las variables, usaremos un mapa de color. Para ello, necesitamos representar la fecha, el único objeto que no es numérico, como un datetime.

In [8]:
sotavento["datetime"] = pd.to_datetime(sotavento["datetime"])

plt.figure(figsize= (20, 12))
correlation = sotavento.corr()

for col in correlation.columns:
	correlation[col] = correlation[col].round(2)

sns.heatmap(correlation, annot=True, cmap="YlGnBu")
plt.show()

NameError: name 'sotavento' is not defined

### **Existencia de columnas constantes**

Para comprobar la existencia de columnas constantes. Debido a la naturaleza de los datos, se considerará que dos valores son distintos si difieren en más de un 1% del valor más grande. A través del siguiente código vemos que **no existen columnas constantes**.

In [ ]:
for j in range(0, 1, 1):
 same_date = True
 same_day = True
 i = 0
 while((same_date or same_day) and i < len(sotavento)-1):
  if (sotavento.iloc[i, j] != sotavento.iloc[i+1,j]):
   same_date = False
  if (sotavento.iloc[i, j][0:10] != sotavento.iloc[i+1,j][0:10]):
   same_day = False
  i += 1
 if same_date:
  print("The datetime values", str(j), "are constant.")
 if same_day:
  print("The values", str(j), "are from the same day")

for j in range(1, len(sotavento.columns), 1):
 same = True
 i = 0
 while (same and i < len(sotavento)-1):
  if (sotavento.iloc[i+1, j] < 0.99*sotavento.iloc[i, j] or sotavento.iloc[i+1,j] > 1.01*sotavento.iloc[i,j]):
   same = False
  i += 1
 if same:
  print("The values of the column", str(j), "are constant.")

# **EDA - 4.Tipo de problema**
Tras estos análisis, debemos instanciar este problema como un problema de **Clasificación** o de **Regresión**. Lo primero que debemos saber para realizar esta clasificación, es saber la variable que tenemos que predecir, es decir, **energía**. Con los bloques de código anteriores, hemos visto que la energía es un valor discreto, por lo que podemos asumir que el modelo intenta resolver un problema de **Regresión**.

Por esto, podemos decir, adicionalmente, que al estimar un valor según unas variables, se trata de un problema del tipo aprendizaje **atributo-valor**. Además, como tenemos los valores de **energía**, podemos decir que se trata de un problema de aprendizaje supervisado.

Una vez finalizado con el EDA, abordaremos el diseño y elección del modelo en el siguiente apartado.



# **Modelo - 1.Evaluaciones *outer* e *inner* y métricas**
En este apartado, decidiremos como llevar a cabo las evaluaciones *outer* e *inner* sobre nuestros datos. Además, pensaremos en las métricas de evaluación que usaremos para estimar el éxito de nuestro modelo.

## **Evaluaciones *outer* e *inner***

Lo primero a decidir para el modelo es como realizaremos las evaluaciones *inner* y *outer* para poder elegir los mejores hiperparámetros. Para ello, tenemos que pensar en los datos que tenemos.

Sabemos que tenemos que predecir un valor discreto, **energía**, y que los datos están ordenados cronológicamente, lo cual es muy importante para decidir la evaluación *outer*. Debido a su orden, la mejor opción para realizar la evaluación outer es dividir los datos usando un *time-split*.

Esto se debe a que queremos los datos de un año para predecir los siguientes, datos que están muy correlados entre sí, y un *cross-validator* que haga un split de los datos según series temporales es el que mejor se adecúa a esta situación. Si queremos hacer una métrica más exacta, será mejor dividir entre meses, pero eso lo decidiremos en la implementación del modelo.

## **Métricas para la evaluación**
Para poder realizar la estimación de la forma más adecuada, es necesario tener en cuenta que el problema es un problema de *Regresión*. Sabiendo esto, debemos elegir una métrica que tenga en cuenta la distancia entre el punto predicho y el punto real.

Hay múltiples opciones que podemos considerar:

- Mean Squared Error (MSE)
- Mean Absolure Error (MAE)
- Relative Mean Squared Error (RMSE)
- Relative Mean Absolute Error (RMAE)

Dada la resistencia del MAE a los valores mal predichos, e incluso a los outliers, parece ser la opción más robusta. Además, para evitar el problema de la escala de la variable respuesta, usaremos un regresor de tipo *dummy*, con el cual comparar las predicciones que hagamos. Al usar el MAE como métrica, debemos usar el *Median dummy regressor*, el cual usará la mediana de la variable respuesta.